## TASK : Data Collection and Integration

In [17]:
import pandas as pd
import numpy as np

### Identify and access relevant data sources:
- Traffic data: This data is provided to you in the dataset.
- Weather data: Collect historical weather data from meteorological services, including temperature, precipitation, humidity, and wind speed.
- Event data: Gather information on special events (sports events, concerts, public holidays) from local event calendars and news sources.

In [18]:
traffic = pd.read_csv('final_df.csv')
events = pd.read_csv('holidays.csv')
weather = pd.read_csv('weather_data.csv', index_col='Unnamed: 0')

In [19]:
events.drop(['Unnamed: 0'], axis=1, inplace=True)


In [20]:
events.head()

,holidays,date
0,Naraka Chaturdasi,2015-11-10
1,Diwali/Deepavali,2015-11-11
2,Govardhan Puja,2015-11-12
3,Bhai Duj,2015-11-13
4,Chhat Puja (Pratihar Sashthi/Surya Sashthi),2015-11-17


In [21]:
weather = weather.reset_index().drop(['index'],axis=1)
weather.head()

,date,time,humidity,weather,temperature
0,2015-11-01,01:00:00,75.0,broken clouds,295.935188
1,2015-11-01,02:00:00,82.0,mist,293.860000
2,2015-11-01,03:00:00,87.0,overcast clouds,293.104524
3,2015-11-01,04:00:00,93.0,light rain,292.280000
4,2015-11-01,05:00:00,77.0,overcast clouds,291.650000


In [22]:
traffic = traffic.drop(['Unnamed: 0'], axis=1)

In [23]:
traffic.head()

,Junction,Date,Hour,Vehicles,hour_of_the_day,day_of_the_week,month,PreviousHourVehicles,IsWeekend
0,1,2015-01-11,0,-0.375489,0,6,1,NaN,True
1,1,2015-01-11,1,-0.471875,1,6,1,-0.375489,True
2,1,2015-01-11,2,-0.616454,2,6,1,-0.471875,True
3,1,2015-01-11,3,-0.761034,3,6,1,-0.616454,True
4,1,2015-01-11,4,-0.664648,4,6,1,-0.761034,True


In [24]:
# converting kelvin to farhenheit
weather['temperature'] = (weather['temperature'] - 272.15)
weather['temperature'] = (weather['temperature'] * 1.8) + 32
weather.head()

,date,time,humidity,weather,temperature
0,2015-11-01,01:00:00,75.0,broken clouds,74.813338
1,2015-11-01,02:00:00,82.0,mist,71.078000
2,2015-11-01,03:00:00,87.0,overcast clouds,69.718144
3,2015-11-01,04:00:00,93.0,light rain,68.234000
4,2015-11-01,05:00:00,77.0,overcast clouds,67.100000


In [25]:
weather['date'] = pd.to_datetime(weather['date'])
events['date'] = pd.to_datetime(events['date'])

In [26]:
weather.drop(['time'], axis=1, inplace=True)

In [27]:
traffic = traffic.rename(columns={'Date': 'date'})

In [28]:
traffic['date'] = pd.to_datetime(traffic['date'])

### 2. Integrate data from various sources:
- Develop a data integration pipeline to merge traffic, weather, and event data into a unified dataset.
- Ensure data is synchronized based on timestamps to align traffic data with corresponding weather conditions and events.


In [29]:
# merging traffic and weather data
df = pd.merge(left=traffic, right=weather, on='date')

In [30]:
events['holidays'] = 1

In [31]:
# merging combined date with events now
merged_df = pd.merge(left=df, right=events, on='date', how='left')

In [32]:
merged_df.head()

,Junction,date,Hour,Vehicles,hour_of_the_day,day_of_the_week,month,PreviousHourVehicles,IsWeekend,humidity,weather,temperature,holidays
0,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,88.0,haze,75.218000,1.0
1,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,85.0,overcast clouds,74.877599,1.0
2,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,83.0,overcast clouds,74.426000,1.0
3,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,80.0,overcast clouds,74.537863,1.0
4,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,78.0,overcast clouds,74.678000,1.0



### 3. Handle data quality issues:
- Clean the dataset by removing duplicates, handling missing values (imputation or removal), and correcting inconsistencies.
- Normalize or standardize data to bring different variables to a common scale.

In [33]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056960 entries, 0 to 1056959
Data columns (total 13 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   Junction              1056960 non-null  int64         
 1   date                  1056960 non-null  datetime64[ns]
 2   Hour                  1056960 non-null  int64         
 3   Vehicles              1056960 non-null  float64       
 4   hour_of_the_day       1056960 non-null  int64         
 5   day_of_the_week       1056960 non-null  int64         
 6   month                 1056960 non-null  int64         
 7   PreviousHourVehicles  1056912 non-null  float64       
 8   IsWeekend             1056960 non-null  bool          
 9   humidity              1056960 non-null  float64       
 10  weather               1056960 non-null  object        
 11  temperature           1056960 non-null  float64       
 12  holidays              198144 non-null   fl

In [34]:
merged_df.isnull().mean()*100

Junction                 0.000000
date                     0.000000
Hour                     0.000000
Vehicles                 0.000000
hour_of_the_day          0.000000
day_of_the_week          0.000000
month                    0.000000
PreviousHourVehicles     0.004541
IsWeekend                0.000000
humidity                 0.000000
weather                  0.000000
temperature              0.000000
holidays                81.253406
dtype: float64

In [35]:
# handling missing values of holidays column
merged_df['holidays']= merged_df['holidays'].fillna(0)

In [36]:
merged_df = merged_df.dropna()

In [37]:
merged_df.head()

,Junction,date,Hour,Vehicles,hour_of_the_day,day_of_the_week,month,PreviousHourVehicles,IsWeekend,humidity,weather,temperature,holidays
0,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,88.0,haze,75.218000,1.0
1,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,85.0,overcast clouds,74.877599,1.0
2,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,83.0,overcast clouds,74.426000,1.0
3,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,80.0,overcast clouds,74.537863,1.0
4,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,78.0,overcast clouds,74.678000,1.0


In [38]:
# handling categorical column i.e. weather
merged_df['weather'].value_counts()

weather
sky is clear                        383640
overcast clouds                     148320
broken clouds                        99904
mist                                 93008
light rain                           79992
scattered clouds                     72740
few clouds                           71516
haze                                 42594
light intensity drizzle              16152
fog                                  15710
moderate rain                        13440
proximity thunderstorm                8040
heavy intensity rain                  4320
thunderstorm                          1920
drizzle                               1800
thunderstorm with light rain          1320
thunderstorm with heavy rain           864
squalls                                504
light snow                             432
thunderstorm with rain                 216
proximity thunderstorm with rain       192
thunderstorm with light drizzle        144
heavy intensity drizzle                 72
ver

In [39]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

merged_df['weather'] = le.fit_transform(merged_df['weather'])

In [40]:
# Normalize or standardize data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

merged_df[['humidity', 'temperature']] = scaler.fit_transform(merged_df[['humidity', 'temperature']])

In [41]:
merged_df.head()

,Junction,date,Hour,Vehicles,hour_of_the_day,day_of_the_week,month,PreviousHourVehicles,IsWeekend,humidity,weather,temperature,holidays
0,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,0.918215,4,0.320519,1.0
1,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,0.778293,12,0.295681,1.0
2,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,0.685012,12,0.262728,1.0
3,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,0.545090,12,0.270891,1.0
4,1,2015-11-11,0,0.251022,0,5,11,-0.038137,False,0.451808,12,0.281116,1.0


In [42]:
merged_df = merged_df.drop(['Hour'], axis=1)

In [43]:
merged_df.head()

,Junction,date,Vehicles,hour_of_the_day,day_of_the_week,month,PreviousHourVehicles,IsWeekend,humidity,weather,temperature,holidays
0,1,2015-11-11,0.251022,0,5,11,-0.038137,False,0.918215,4,0.320519,1.0
1,1,2015-11-11,0.251022,0,5,11,-0.038137,False,0.778293,12,0.295681,1.0
2,1,2015-11-11,0.251022,0,5,11,-0.038137,False,0.685012,12,0.262728,1.0
3,1,2015-11-11,0.251022,0,5,11,-0.038137,False,0.545090,12,0.270891,1.0
4,1,2015-11-11,0.251022,0,5,11,-0.038137,False,0.451808,12,0.281116,1.0


In [47]:
merged_df[:1000].to_csv('merged_df.csv')

In [48]:
merged_df.shape

(1056912, 12)